In [4]:
#!pip install selenium

notebook 5.4.0 requires ipykernel, which is not installed.
jupyter 1.0.0 requires ipykernel, which is not installed.
jupyter-console 5.2.0 requires ipykernel, which is not installed.
ipywidgets 7.1.1 requires ipykernel>=4.5.1, which is not installed.
You are using pip version 10.0.0, however version 10.0.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [45]:
#Imports:
import selenium
from selenium import webdriver
import time
import pandas as pd
import re
import tqdm
import datetime

In [42]:
#Constants:
URL_TO_SCRAPE = "https://www.instagram.com/antlerofficial"
    #Scraper delays(s):
DELAY_GETPOSTDATA = 0.1
DELAY_GETALLPOSTDATA = 0.1
DELAY_SCROLLER = 1
DELAY_COMMENT_EXPANDER = 0.1

In [51]:
def scrollPageToBottomAndFindPostLinks():
    allPosts = []
    
    prevHeight = 0
    newHeight = 1

    while prevHeight != newHeight:
        prevHeight = int(driver.execute_script("return document.body.scrollHeight;"))
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight)")
        time.sleep(DELAY_SCROLLER)
        allPosts.append(findPostLinks(driver))
        newHeight = int(driver.execute_script("return document.body.scrollHeight;"))
        
    return allPosts; 

def findPostLinks(driver):
    posts = []
    src = driver.page_source
    src_splitted = src.split("</div>")

    for entry in src_splitted:
        postRegex = re.findall("(<a href=\")(.*)(\?taken-by=\w*\">)", entry)
        if len(postRegex) > 0:
            posts.append(postRegex[0][1])
    return posts

def getPostData(driver):
    time.sleep(DELAY_GETPOSTDATA)
    postData = []
    expandAllComments(driver)
    article = driver.find_element_by_tag_name("article")
    post_link = driver.current_url
    image_link = driver.find_elements_by_tag_name('img')[1].get_attribute('src')
    author = article.text.splitlines()[0]
    authors, comments = getAllCommentsFromArticle(driver)
    #Getting likes
    try:
        likes = driver.find_element_by_xpath("//*[@id=\"react-root\"]/section/main/div/div/article/div[2]/section[2]/div/span/span").text#driver.find_element_by_tag_name('span').text.splitlines()[6].replace(" likes", "")
    except Exception as e:
        print("Unable to get likes from post: " + post_link + "\n Trying different approach...")
        try:
            likes = (len(driver.find_element_by_xpath("//*[@id=\"react-root\"]/section/main/div/div/article/div[2]/section[2]/div").text.split(",")) + 1)
        except Exception as Ex:
            print("Different approach didn't work. Value of likes field will be \"Exception\"")
            likes = "Exception"
    #Getting date
    try:
        date = driver.find_element_by_xpath("//*[@id=\"react-root\"]/section/main/div/div/article/div[2]/div[2]/a/time").text#driver.find_element_by_tag_name('article').text.splitlines()[len(driver.find_element_by_tag_name('article').text.splitlines()) - 2]
    except Exception as e:
        print("Unable to get date from post: " + post_link)
        date = "Exception"
    if ("," not in date):
        date = date + ", " + str(datetime.datetime.now().year)

    firstRun = True
    for a, c in zip(authors, comments):
        if firstRun:
            firstRun = False
            postData.append({"post_link" : post_link, "image_link" : image_link, "post_author" : a, "post" : c, "likes": likes, "date": date, "comment": "", "comment_author": ""})
        else:
            postData.append({"post_link" : "", "image_link" : "", "post_author" : "", "post" : "", "likes": "", "date": "", "comment": c, "comment_author": a})
    return postData

def getAllCommentsFromArticle(driver):
    #Posts description and authors name is a very first comment's content and authors name
    authors = []
    comments = []
    article = driver.find_element_by_tag_name("article")
    comment = article.find_elements_by_tag_name("li")
    for com in comment:
        #print("\n" + com.find_element_by_tag_name("a").text + ", post: " + com.find_element_by_tag_name("span").text)
        authors.append(com.find_element_by_tag_name("a").text)
        comments.append(com.find_element_by_tag_name("span").text)
    return authors, comments

def expandAllComments(driver):
    try:
        el = driver.find_element_by_xpath("//*[@id=\"react-root\"]/section/main/div/div/article/div[2]/div[1]/ul/li[2]/a")
        while (el.text == "Load more comments" or "View all" in el.text):
            el = driver.find_element_by_xpath("//*[@id=\"react-root\"]/section/main/div/div/article/div[2]/div[1]/ul/li[2]/a")
            if (el.text == "Load more comments" or "View all" in el.text):
                el.click()
            time.sleep(DELAY_COMMENT_EXPANDER)
    except Exception as e:
        print("expandAllComments() - expanded")

In [3]:
if __name__ == "__main__":
    driver = webdriver.Chrome()
    driver.get(URL_TO_SCRAPE)
    allPosts = scrollPageToBottomAndFindPostLinks()
    
    #Concatenate all lists in one
    allPosts = sum(allPosts, [])
    #Remove dupes from posts
    allPosts = list(set(allPosts))
    allPosts = ["https://www.instagram.com" + x for x in allPosts]
    print("Amount of links to posts scraped: " + str(len(allPosts)))

Amount of links to posts scraped: 694


In [52]:
if __name__ == "__main__":
    allPostData = []
    #driver = webdriver.Chrome()
    for link in tqdm.tqdm(allPosts):
        driver.get(link)
        time.sleep(DELAY_GETALLPOSTDATA)
        allPostData.append(getPostData(driver))
    #Export to file:
    allPostData = sum(allPostData, [])
    df = pd.DataFrame.from_dict(allPostData)
    df = df.reindex(['post_link', "image_link", "post", "date", "post_author", "likes", "comment", "comment_author"], axis=1)
    df.to_excel("allposts_current.xlsx")



  0%|                                                                                          | 0/694 [00:00<?, ?it/s]

  0%|                                                                                  | 1/694 [00:01<17:43,  1.53s/it]

  0%|▏                                                                                 | 2/694 [00:03<17:31,  1.52s/it]

expandAllComments() - expanded




  0%|▎                                                                                 | 3/694 [00:04<16:02,  1.39s/it]

  1%|▍                                                                                 | 4/694 [00:05<15:33,  1.35s/it]

expandAllComments() - expanded




  1%|▌                                                                                 | 5/694 [00:06<14:54,  1.30s/it]

  1%|▋                                                                                 | 6/694 [00:08<15:20,  1.34s/it]

  1%|▊                                                                                 | 7/694 [00:09<15:41,  1.37s/it]

KeyboardInterrupt: 

In [50]:

#df['date'] = pd.to_datetime(df.date)
#df.sort('date') # This now sorts in date order



In [ ]:
driver.quit()